# Interpreters

> infix to postix, interpreting formulas

In [ ]:
import sys
from functools import reduce

def binary2nary(f):
    return lambda *args: reduce(f, args)

def curry(f, x):  # binds first arg
    return lambda *xs: f(*(x,) + xs)

def rcurry(f, x):  # binds last arg
    return lambda *xs: f(*xs + (x,))

def flip(f):
    return lambda x, y: f(y, x)  # flips args

def compose(f, g):
    return lambda *x: f(g(*x))

def negate(p):
    return lambda x: not p(x)

def unary_and(p, q):
    return lambda x: p(x) and q(x)

def unary_or(p, q):
    return lambda x: p(x) or q(x)

## Infix to postfix

In [ ]:
def brackets_ok(formula: str) -> bool:
    """
    :param formula: a formula containing round brackets
    :return: True if the brackets match
    """
    bc = 0  ## bracket count
    for c in formula:
        if c == '(':
            bc += 1
        elif c == ')':
            bc -= 1
            if bc < 0:  # bc must never be negative
                return False
    return bc == 0  # bc must be 0

def strip_brackets(formula: str) -> str:
    """
    :param formula: a formula
    :return: formula without blanks and redundant outer brackets stripped
    """
    while formula[0] == '(' and formula[-1] == ')' and brackets_ok(formula[1:-1]):
        formula = formula[1:-1]
    return formula

def split(formula: str, operators: dict) -> tuple:
    """
    :param formula: an infix formula
    :param operators: a dict of operators
    :return:
        a) the formula's first part
        b) the innermost operator (lowest priority)
        c) the formula's second part
    """
    bracket_count = 0  # counts brackets
    min_weight = sys.maxsize  # weight of lowest priority op
    min_idx = -1  # index of this op

    for i, c in enumerate(formula):
        if c == '(':
            bracket_count += 1
        elif c == ')':
            bracket_count -= 1
        elif c in operators:
            arity, prio = operators[c]
            w = prio + bracket_count
            if min_weight >= w:
                min_weight, min_idx = w, i
    if min_idx < 0:  # no operator
        return formula, None, None
    else:
        return formula[:min_idx], formula[min_idx], formula[min_idx + 1:]

## A Simple Grammar

In [ ]:
# the dict operators contains all operators with
# 1) arity (= 1 or 2) and
# 2) priority (= 1, 2 or 3; 1 = low, 3 = high)

operators = {'+': (2, 1), '-': (2, 1), '*': (2, 2), '/': (2, 2), '~': (1, 3)}
# print(split(formula, operators))


def infix2postfix(formula: str, operators: dict) -> str:
    """
    :param formula: an infix formula
    :param operators: a dictionary of operators
    :return: formula as postfix
    """
    formula = formula.strip()
    if not formula:
        return formula
    formula = strip_brackets(formula)
    left, op, right = split(formula, operators)
    if op is None:
        return formula
    left = infix2postfix(left, operators)
    right = infix2postfix(right, operators)
    return left + right + op

# formula = "(((a) + (b - z)) + c) * d"
formula = "a +~b-c * d + (z + ~t - ~u) "
# formula = "a - b + c"
print(infix2postfix(formula, operators))

ab~+cd*-zt~+u~-+


In [ ]:
def translate(formula, namespace):
    """
    translate a postfix-formula into an executable predicate
    operators are AND, OR, NOT
    namespace contains predicates
    """

    stack = []
    for token in formula.split():
        if token == "AND":
            p = stack.pop()
            q = stack.pop()
            stack.append(unary_and(q, p))
        elif token == "OR":
            p = stack.pop()
            q = stack.pop()
            stack.append(unary_or(q, p))
        elif token == "NOT":
            p = stack.pop()
            stack.append(negate(p))
        else:
            stack.append(namespace[token])
    if len(stack) != 1:
        raise ValueError('bad formula')
    return stack.pop()

def odd(x):
    return bool(x % 2)

even = negate(odd)

namespace = {'a' : odd, 'b' : even}
formula = 'a b AND'

p = translate(formula, namespace)
print(p(1))

False


##  Compiling FORTH

todo


